In [1]:
"""
    This script splits the JSON into multiple files, according to ROWID_SPLITS use to separate round 1 and round 2 data.
"""

'\n    This script splits the JSON into multiple files, according to ROWID_SPLITS use to separate round 1 and round 2 data.\n'

In [2]:
import os
import pandas as pd
import numpy as np
import json
import re

# Define the name of the data file
DATA_FILE = "all_data_0309_parsed.csv"

EXPORT_DIR = "../healsl_rd1to2_rapid_gpt3_v2b_2024_03_11"


if not os.path.exists(EXPORT_DIR):
    print(f"Creating directory: {EXPORT_DIR}")
    os.makedirs(EXPORT_DIR)


/var/folders/jk/dwx9rt0d03d6hmp1hj2xtrdr0000gp/T/ipykernel_39468/3717094835.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv(DATA_FILE)

In [4]:
df[df['round'] == "rd1"].to_csv(f"{EXPORT_DIR}/healsl_rd1_rapid_gpt3_v2b.csv", index=False)
df[df['round'] == "rd2"].to_csv(f"{EXPORT_DIR}/healsl_rd2_rapid_gpt3_v2b.csv", index=False)

